<a href="https://colab.research.google.com/github/shubhamnawani99/stocks-analysis/blob/master/Module_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#1.1 Importing the csv file of the stock you have been allotted using 'pd.read_csv()' function into a dataframe.
#import the necessary files
import pandas as pd
import numpy as np

In [2]:
#path for the given .csv file
path = 'https://raw.githubusercontent.com/shubhamnawani99/stocks-analysis/master/INTC.csv'

#No 'Series' column in the chosen shares file, thus no need for pre-processing
#stocks = stocks[stocks.Series == 'EQ']

#defining the dataframe
stocks = pd.read_csv(path)
stocks.head()

#stocks.describe()
#stocks.tail()

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity
0,"Oct 24, 2019",52.23,52.11,52.42,51.44,25.65M
1,"Oct 23, 2019",51.72,51.37,51.96,51.20,15.96M
2,"Oct 22, 2019",52.01,51.90,52.32,51.89,13.35M
3,"Oct 21, 2019",52.13,51.52,52.17,51.34,13.12M
4,"Oct 18, 2019",51.36,51.75,51.82,51.03,16.98M


In [3]:
#1.2 Calculate the maximum, minimum and mean price for the last 90 days
#Using loc[] and summary funcations - max(), min() and mean() to calculate the max, min and mean of the closing price for the last 90 days

values = {'Max':stocks.loc[:89,'Close Price'].max(),'Min':stocks.loc[:89,'Close Price'].min(),'Mean':stocks.loc[:89,'Close Price'].mean()}
values

{'Max': 53.01, 'Mean': 49.44277777777777, 'Min': 44.96}

In [4]:
#1.3 Analyse the data types for each column of the Stocks dataframe
stocks.dtypes

Date                      object
Close Price              float64
Open Price               float64
High Price               float64
Low Price                float64
Total Traded Quantity     object
dtype: object

In [5]:
#now to convert the Date from type 'object' to type 'datetime64(ns)'
stocks.Date = stocks.Date.apply(pd.to_datetime)

#subtracting max Date value with the min Date value
stocks.Date.max() - stocks.Date.min()

Timedelta('729 days 00:00:00')

In [6]:
#to view the if the changes have been reflected back in the dataframe 
stocks.dtypes

Date                     datetime64[ns]
Close Price                     float64
Open Price                      float64
High Price                      float64
Low Price                       float64
Total Traded Quantity            object
dtype: object

In [0]:
#1.4 In a separate array , calculate the monthwise VWAP (Volume Weighted Average Price ) of the stock.
#Converting Total Volume from object into float values
if stocks['Total Traded Quantity'].dtype != 'float64':
  stocks['Total Traded Quantity'] = stocks['Total Traded Quantity'].replace({'M': '*1e6'}, regex=True).map(pd.eval).astype(float)

In [0]:
#Extracting Months and Years
stocks['Month'] = pd.DatetimeIndex(stocks['Date']).month
stocks['Year'] = pd.DatetimeIndex(stocks['Date']).year
group = stocks.groupby(['Month', 'Year'])

#Calculating VWAP
stocks['VWAP'] = (np.cumsum(stocks['Close Price'] * stocks['Total Traded Quantity']) / np.cumsum(stocks['Total Traded Quantity']))

In [9]:
#1.5 Calculation of average price and profit/loss 
def avgPrice(N):
    res =  stocks['Close Price'].iloc[-N:]
    return res.mean()
print("Average prices for last N days are as follows:")
print("Last 1 week",avgPrice(5))
print("Last 2 weeks",avgPrice(10))
print("Last 1 month",avgPrice(20))
print("Last 3 months",avgPrice(60))
print("Last 6 months",avgPrice(120))
print("Last 1 year",avgPrice(240))

def profLoss(N):
    diff = (stocks['Close Price'].tail(N).iloc[N-1] - stocks['Close Price'].tail(N).iloc[0])
    if diff < 0 :
        loss = -(diff)
        loss_value = (loss/stocks['Close Price'].tail(N).iloc[N-1])*100
        return loss_value
    if diff > 0 :
        profit = diff
        profit_value = (profit/data['Close Price'].tail(N).iloc[N-1])*100
        return profit_value

print("\nThe Profit/Loss is as follows: ")
print("Last 1 week",profLoss(5))
print("Last 2 weeks",profLoss(10))
print("Last 1 month",profLoss(20))
print("Last 3 months",profLoss(60))
print("Last 6 months",profLoss(120))
print("Last 1 year",profLoss(240))

Average prices for last N days are as follows:
Last 1 week 43.278
Last 2 weeks 45.001999999999995
Last 1 month 45.27550000000001
Last 3 months 44.897000000000006
Last 6 months 46.95966666666663
Last 1 year 48.83770833333332

The Profit/Loss is as follows: 
Last 1 week 11.549779303580188
Last 2 weeks 14.713094654242276
Last 1 month 10.201078960274636
Last 3 months 12.187346738597348
Last 6 months 31.46150073565473
Last 1 year 15.326140264835702


In [10]:
#1.6 Calculation of daily change in percentages
stocks['Day_Perc_Change'] = stocks['Close Price'].pct_change().fillna(0)
stocks

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,VWAP,Day_Perc_Change
0,2019-10-24,52.23,52.11,52.42,51.44,25650000.0,10,2019,52.230000,0.000000
1,2019-10-23,51.72,51.37,51.96,51.20,15960000.0,10,2019,52.034384,-0.009765
2,2019-10-22,52.01,51.90,52.32,51.89,13350000.0,10,2019,52.028461,0.005607
3,2019-10-21,52.13,51.52,52.17,51.34,13120000.0,10,2019,52.048029,0.002307
4,2019-10-18,51.36,51.75,51.82,51.03,16980000.0,10,2019,51.910682,-0.014771
...,...,...,...,...,...,...,...,...,...,...
498,2017-10-31,45.49,45.20,45.80,44.65,47000000.0,10,2017,48.752448,-0.026119
499,2017-10-30,44.37,44.48,45.19,43.88,48160000.0,10,2017,48.736542,-0.024621
500,2017-10-27,44.40,43.30,45.00,43.10,90030000.0,10,2017,48.707318,0.000676
501,2017-10-26,41.35,40.91,41.58,40.71,38250000.0,10,2017,48.686312,-0.068694


In [11]:
#1.7 Addition of column "Trend"
if ((stocks['Day_Perc_Change'] >= -0.5) & (stocks['Day_Perc_Change'] <= 0.5)).all():
    stocks['Trend'] = 'Slight or No change'
if ((stocks['Day_Perc_Change'] >= 0.5) & (stocks['Day_Perc_Change'] <= 1)).all():
    stocks['Trend'] = 'Slight positive'
if ((stocks['Day_Perc_Change'] <= -0.5) & (stocks['Day_Perc_Change'] >= -1)).all():
    stocks['Trend'] = 'Slight negative'
if ((stocks['Day_Perc_Change'] >= 1) & (stocks['Day_Perc_Change'] <= 3)).all():
    stocks['Trend'] = 'Positive' 
if ((stocks['Day_Perc_Change'] <= -1) & (stocks['Day_Perc_Change'] >= -3)).all():
    stocks['Trend'] = 'Negative'
if ((stocks['Day_Perc_Change'] >= 3) & (stocks['Day_Perc_Change'] <= 7)).all():
    stocks['Trend'] = 'Among top gainers'
if ((stocks['Day_Perc_Change'] <= -3) & (stocks['Day_Perc_Change'] >= -7)).all():
    stocks['Trend'] = 'Among top losers'
if (stocks['Day_Perc_Change'] > 7).all():
    stocks['Trend'] = 'Bull run' 
if (stocks['Day_Perc_Change'] < -7).all():
    stocks['Trend'] = 'Bear drop'
stocks

,Date,Close Price,Open Price,High Price,Low Price,Total Traded Quantity,Month,Year,VWAP,Day_Perc_Change,Trend
0,2019-10-24,52.23,52.11,52.42,51.44,25650000.0,10,2019,52.230000,0.000000,Slight or No change
1,2019-10-23,51.72,51.37,51.96,51.20,15960000.0,10,2019,52.034384,-0.009765,Slight or No change
2,2019-10-22,52.01,51.90,52.32,51.89,13350000.0,10,2019,52.028461,0.005607,Slight or No change
3,2019-10-21,52.13,51.52,52.17,51.34,13120000.0,10,2019,52.048029,0.002307,Slight or No change
4,2019-10-18,51.36,51.75,51.82,51.03,16980000.0,10,2019,51.910682,-0.014771,Slight or No change
...,...,...,...,...,...,...,...,...,...,...,...
498,2017-10-31,45.49,45.20,45.80,44.65,47000000.0,10,2017,48.752448,-0.026119,Slight or No change
499,2017-10-30,44.37,44.48,45.19,43.88,48160000.0,10,2017,48.736542,-0.024621,Slight or No change
500,2017-10-27,44.40,43.30,45.00,43.10,90030000.0,10,2017,48.707318,0.000676,Slight or No change
501,2017-10-26,41.35,40.91,41.58,40.71,38250000.0,10,2017,48.686312,-0.068694,Slight or No change


In [12]:
#1.8 Calculation of average and median values of 'Total Traded Quantity' for each types of 'Trend'
AvgMedian = {'Average':stocks.groupby(stocks.Trend).mean()['Total Traded Quantity'],'Median':stocks.groupby(stocks.Trend).median()['Total Traded Quantity']}
print(AvgMedian)

{'Average': Trend
Slight or No change    2.668302e+07
Name: Total Traded Quantity, dtype: float64, 'Median': Trend
Slight or No change    23800000.0
Name: Total Traded Quantity, dtype: float64}


In [0]:
#1.9 Saving dataframe with revised columns to week2.csv
stocks.to_csv("week2.csv")